In [ ]:
import torch
import statistics
from statistics import mean
import csv
from sklearn import metrics
from copy import deepcopy
import numpy as np
from torch_geometric.nn import MessagePassing
from torch.nn.parameter import Parameter

In [ ]:
class evalaution_of_xmethods_fidelity_plus(object):
    def __init__(self, a_trained_model, test_data):
        super(evalaution_of_xmethods_fidelity_plus, self).__init__()
        self.a_trained_model = a_trained_model
        self.test_data = test_data

    def is_salient(self, score, importance_threshold):
        if importance_threshold == score == 0:
            return True
        if importance_threshold == score == 1:
            return False
        if importance_threshold < score:
            return True
        else:
            return False

    def Compute_ROC_AUC(self, your_model, your_dataset):
        preds = []
        reals = []

        your_model.eval()
        for batched_data in your_dataset:
            if your_model.__class__.__name__ == "GCN_plus_GAP":
                Output_of_Hidden_Layers, pooling_layer_output, ffn_output, prediction = your_model(batched_data)
            elif your_model.__class__.__name__ == "DGCNN_Model":
                final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                concatination_list_of_poolings, prediction_output_not_softed, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "GIN_Model":
                mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, prediction = your_model(batched_data, None)
            else:
                print("Model Name is not valid.")

            # preds.extend(torch.argmax(prediction, dim=1).tolist())
            preds.extend(prediction.tolist())
            reals.extend(batched_data.y.tolist())

        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)

        auc_roc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")

        return auc_roc

    def drop_important_node_features(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                for k in range(len(attribution_scores[i][j])):
                    if self.is_salient(attribution_scores[i][j][k], importance_threshold):
                        sample_graph.x[j][k] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def drop_important_nodes(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                if self.is_salient(attribution_scores[i][j], importance_threshold):
                    sample_graph.x[j][:] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def Fidelity_node_features(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_important_node_features(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def Fidelity_node(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_important_nodes(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def drop_important_edges(self, your_dataset, importance_threshold, Edge_Masks_Dropped):
        attribution_scores = []
        Edge_Masks_original = []
        Edge_Masks_Dropped_copy = []
        for tensor in Edge_Masks_Dropped:
            Edge_Masks_original.append(deepcopy(tensor.detach()))
            Edge_Masks_Dropped_copy.append(deepcopy(tensor.detach()))

        for edge_mask in Edge_Masks_Dropped_copy:

            importance_indices = edge_mask > importance_threshold * (max(edge_mask)-min(edge_mask))
            edge_mask[importance_indices] = 0

        return Edge_Masks_original, Edge_Masks_Dropped_copy

    def Fidelity_edge(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, False)
        Edge_Masks_original, Edge_Masks_Dropped = self.drop_important_edges(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, Edge_Masks_Dropped)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def normalize_saliency_node_features_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                node_gradients = []
                for dim in node_grads:
                    node_gradients.append((dim-min(node_grads))/(max(node_grads)-min(node_grads)))
                new_gradients.append(node_gradients)
            Graphs_new_gradients.append(new_gradients)
        return Graphs_new_gradients

    def normalize_saliency_node_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                val = (node_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                new_gradients.append(val)
            Graphs_new_gradients.append(new_gradients)
        return Graphs_new_gradients

    def clear_masks(self, model):

        for module in model.modules():
            if isinstance(module, MessagePassing):
                module.explain = False
                module._edge_mask = None
                module._loop_mask = None
                module._apply_sigmoid = True
        return module

    def apply_masks(self, model, mask, edge_index, apply_sigmoid):
        loop_mask = edge_index[0] != edge_index[1]

        for module in model.modules():
            if isinstance(module, MessagePassing):

                if (not isinstance(mask, Parameter)
                        and '_edge_mask' in module._parameters):
                    mask = Parameter(mask)

                module.explain = True
                module._edge_mask = mask
                module._loop_mask = loop_mask
                module._apply_sigmoid = apply_sigmoid

    def Compute_ROC_AUC_edge(self, your_model, your_dataset, masked):
        preds = []
        reals = []

        if masked == False:
            your_model.eval()
            for batched_data in your_dataset:
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
        else:
            your_model.eval()
            for batched_data, edge_mask in zip(your_dataset, masked):
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    self.apply_masks(your_model, edge_mask, batched_data.edge_index, apply_sigmoid=True)
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                    self.clear_masks(your_model)
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())


        # preds = torch.cat(preds)
        # preds, max_idxs = torch.max(preds[:], dim=1)
        # print("preds: ", preds)
        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)
        roc_auc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")
        return roc_auc

    def normalize_saliency_edge_based(self, saliency_maps):
        Graphs_new_gradients = []

        for graph_grads in saliency_maps:
            for edge_grads in graph_grads:
                edge_grads = (edge_grads-torch.min(graph_grads))/(torch.max(graph_grads)-torch.min(graph_grads)) if (torch.max(graph_grads)-torch.min(graph_grads)) != 0 else 0
            Graphs_new_gradients.append(graph_grads)

        return Graphs_new_gradients

    def my_fidelity(self, saliencies_for_multiple_classes, importance_threshold, style):
        if style == "Node Feature":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_features_based(value)
                    fid_score = self.Fidelity_node_features(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)

                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style == "Node":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_based(value)
                    fid_score = self.Fidelity_node(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style =="Edge":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_edge_based(value)
                    fid_score = self.Fidelity_edge(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        else:
            print("node based not covered yet")